### Гомологичное моделирование комплекса белка с лигандом

In [1]:
import sys 
import modeller 
import _modeller
import modeller.automodel 
import nglview
import ipywidgets

Запустим Modeller.

In [2]:
env = modeller.environ()
env.io.hetatm = True


                         MODELLER 9.19, 2017/07/19, r11078

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2017 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Linux shadbox 3.2.0-126-generic x86_64
Date and time of compilation

Скачаем белок-заготовку лизоцим радужной форели и последовательность рандомного белока из UniProtKB. Я взяла белок лисовидного поссума. 

In [3]:
! wget http://www.pdb.org/pdb/files/1lmp.pdb 2> /dev/null # чтобы не генерировать много текста
! wget http://www.uniprot.org/uniprot/P51782.fasta 2> /dev/null

Создадим объект выравнивание, добавим последовательность. Заодно добавим в выравнивание белок-заготовку.

In [4]:
alignm = modeller.alignment(env)

alignm.append(file = 'P51782.fasta', align_codes = 'all', alignment_format = 'FASTA')
alignm[0].code = 'no_ligand'

mdl = modeller.model(env, file = '1lmp.pdb')
alignm.append_model(mdl, align_codes = '1lmp', atom_files = '1lmp.pdb')

read_pd_459W> Residue type  NDG not recognized. 'automodel' model building
              will treat this residue as a rigid body.
              To use real parameters, add the residue type to ${LIB}/restyp.lib,
              its topology to ${LIB}/top_*.lib, and suitable forcefield
              parameters to ${LIB}/par.lib.
read_pd_459W> Residue type  NAG not recognized. 'automodel' model building
              will treat this residue as a rigid body.
              To use real parameters, add the residue type to ${LIB}/restyp.lib,
              its topology to ${LIB}/top_*.lib, and suitable forcefield
              parameters to ${LIB}/par.lib.


Делаем выравнивание и сохраняем.

In [5]:
alignm.salign()
alignm.write(file = 'no_ligand_alignment.ali', alignment_format = 'PIR')


SALIGN_____> adding the next group to the alignment; iteration    1


Построим модель.

In [6]:
s = alignm[0]
pdb = alignm[1]

print s.code, pdb.code
print

a = modeller.automodel.automodel(env, 
                                 alnfile = 'no_ligand_alignment.ali', 
                                 knowns = pdb.code, 
                                 sequence = s.code)
a.name = 'mod' + s.code
a.starting_model = 1
a.ending_model = 2 # сделаем всего две модели

orig_stdout = sys.stdout # чтобы уменьшить количество текста, будем писать в файл
with open('output.txt', 'w') as output:
    sys.stdout = output
    a.make()
sys.stdout = orig_stdout

print 'done!'

no_ligand 1lmp

done!


In [7]:
no_ligand = nglview.show_structure_file('no_ligand.B99990001.pdb')
no_ligand

NGLWidget()

А лиганда нет.

In [8]:
lmp = nglview.show_structure_file('1lmp.pdb')
lmp

NGLWidget()

А у форели есть. Последние три остатка.

In [9]:
alignm[1].residues[-3:]

[Residue 1:B (type NDG), Residue 2:B (type NAG), Residue 3:B (type NAG)]

Сделаем новую последовательность. Для обозначения лиганда достаточно добавить три точки в конце.

In [10]:
sequence = ''.join([i.code for i in alignm[0].residues]) + '...'

alignm.append_sequence(sequence)
alignm[2].code = 'with_ligand'

print sequence

MKVLLLLGFIFCSMAAHGKRMERCEFARRIKQLHLDGYHQISLANWVCLAQWESGFDTKATNYNPGDQSTDYGILQINSHYWCDDGKTPHAANECKVRCSELQEDDLVKAVNCAKKIVDQQGIRAWVAWRNKCEGKDLSKYLEGCHL...


Делаем новое выравнивание и запускаем модель с новой последовательностью.

In [11]:
alignm.salign()
alignm.write(file = 'with_ligand_alignment.ali', alignment_format = 'PIR')

s = alignm[2] # заменяем исходную последовательность на новую
pdb = alignm[1] # заготовку оставляем

print s.code, pdb.code
print

a = modeller.automodel.automodel(env, 
                                 alnfile = 'with_ligand_alignment.ali', 
                                 knowns = pdb.code, 
                                 sequence = s.code )
a.name = 'mod' + s.code
a.starting_model = 1
a.ending_model = 2

orig_stdout = sys.stdout 
with open('output.txt', 'w') as output:
    sys.stdout = output
    a.make()
sys.stdout = orig_stdout

print 'done!'


SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    2
with_ligand 1lmp

automodel__W> Topology and/or parameter libraries already in memory. These will
                be used instead of the automodel defaults. If this is not what you
                want, clear them before creating the automodel object with
                env.libs.topology.clear() and env.libs.parameters.clear()
done!


In [12]:
with_ligand = nglview.show_structure_file('with_ligand.B99990001.pdb')
with_ligand

NGLWidget()

Теперь лиганд на месте.